In [1]:
import numpy as np
import torch
import pickle
import os
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
path = r"Y:\Datasets\Fyzio"
exercise = "Wide squat"


In [3]:
with open(os.path.join(path,"X_train",exercise + ".pkl"),"rb") as f:
    x = np.array(pickle.load(f))
    print(x.shape)

(20551, 200, 20)


In [4]:
with open(os.path.join(path,"Y_train",exercise + ".pkl"),"rb") as f:
    y = np.array(pickle.load(f))
    print(y.shape)


(20551, 1)


In [5]:

x_norm = np.zeros_like(x)
means = []
stds = []

for ch in range(x.shape[1]):
    mean = x[:, ch, :].mean()
    std = x[:, ch, :].std()
    x_norm[:, ch, :] = (x[:, ch, :] - mean) / std
    means.append(mean)
    stds.append(std)

# y_min = y.min(axis=0, keepdims=True)
# y_max = y.max(axis=0, keepdims=True)
# y_norm = (y - y_min) / (y_max - y_min)
y_norm = y/100

In [6]:
y_norm_t = torch.from_numpy(y_norm).float()[:,0]

x_norm_t = torch.from_numpy(x_norm).float()
x_norm_t = x_norm_t.permute(0,2,1)
x_train,x_test,y_train,y_test = train_test_split(x_norm_t,y_norm_t,test_size=0.3,random_state=42)
print(y_train)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



tensor([0.8793, 0.5044, 0.0193,  ..., 0.8471, 0.7234, 0.6378])


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

cnn1 = 64
cnn2 = 128
cnn3 = 128
class Net(nn.Module):
    def __init__(self, in_channels,seq_len):
        super(Net,self).__init__()
        self.cnn1 = nn.Conv1d(in_channels, cnn1, kernel_size=3, padding=1)
        self.normalization1=nn.BatchNorm1d(cnn1)
        self.pool1= nn.MaxPool1d(2)
        self.cnn2 = nn.Conv1d(cnn1, cnn2, kernel_size=3, padding=1)
        self.normalization2=nn.BatchNorm1d(cnn2)
        self.pool2= nn.MaxPool1d(2)
        self.cnn3 = nn.Conv1d(cnn2, cnn3, kernel_size=3, padding=1)
        self.normalization3=nn.BatchNorm1d(cnn3)
        self.pool3= nn.MaxPool1d(2)
        self.flatten = nn.Flatten()

        with torch.no_grad():
            dummy = torch.zeros(1, in_channels, seq_len)
            out = self.pool1(F.relu(self.cnn1(dummy)))
            out = self.pool2(F.relu(self.cnn2(out)))
            out = self.pool3(F.relu(self.cnn3(out)))
            flat_dim = out.view(1, -1).size(1)


        self.snn1 = nn.Linear(flat_dim, 64)
        self.dropout = nn.Dropout(p=0.5)
        self.snn2 = nn.Linear(64, 8)
        self.dropout = nn.Dropout(p=0.3)
        self.snn3 = nn.Linear(8, 1)

    def forward(self, x):
        x = self.cnn1(x)
        x = F.relu(x)
        x = self.normalization1(x)
        x = self.pool1(x)
        x = self.cnn2(x)
        x = F.relu(x)
        x = self.normalization2(x)
        x = self.pool2(x)
        x = self.cnn3(x)
        x = F.relu(x)
        x = self.normalization3(x)
        x = self.pool3(x)
        x = self.flatten(x)
        x = self.snn1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.snn2(x)
        x = F.relu(x)
        x = self.dropout(x)
        out = self.snn3(x)
        return out

# in_channels = x_train.shape[1]
# seq_len = x_train.shape[2]
#
# out_features = 1
#
# net = Net(in_channels,seq_len).to(device)
# params = list(net.parameters())
# print(len(params))
# print(params[0].size())


In [8]:
# from torch.utils.data import TensorDataset, DataLoader
#
# train_dataset = TensorDataset(x_train, y_train)
# val_dataset = TensorDataset(x_test, y_test)
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
from sklearn.model_selection import KFold
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_squared_error, r2_score
kf = KFold(n_splits=5, shuffle=True, random_state=42)

X = x_norm_t.cpu().numpy()
Y = y_norm_t.cpu().numpy()

in_channels = x_train.shape[1]
seq_len = x_train.shape[2]
epochs = 100
batch_size = 32

fold_results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"Fold {fold+1}")

    X_train, X_val = X[train_idx], X[val_idx]
    Y_train, Y_val = Y[train_idx], Y[val_idx]

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32).view(-1, 1)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    Y_val_tensor = torch.tensor(Y_val, dtype=torch.float32).view(-1, 1)

    train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, Y_val_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    net = Net(in_channels, seq_len).to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)
    criterion = nn.MSELoss()

    for epoch in range(epochs):
        net.train()
        train_loss = 0.0
        for batch_X, batch_y in train_loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)

            optimizer.zero_grad()
            outputs = net(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
        train_loss /= len(train_loader)


        net.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X = batch_X.to(device)
                batch_y = batch_y.to(device)
                val_outputs = net(batch_X)
                loss = criterion(val_outputs, batch_y)
                val_loss += loss.item()
        val_loss /= len(val_loader)

        if (epoch+1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    net.eval()
    with torch.no_grad():
        preds = net(X_val_tensor.to(device)).numpy().flatten()
        y_true = Y_val_tensor.numpy().flatten()

    mse = mean_squared_error(y_true, preds)
    r2 = r2_score(y_true, preds)
    print(f"Fold {fold+1} - MSE: {mse:.4f}, R²: {r2:.4f}\n")

    fold_results.append((y_true, preds))
mse_l = []
r2_l = []

for y_true, preds in fold_results:
    mse_l.append(mean_squared_error(y_true, preds))
    r2_l.append(r2_score(y_true, preds))

print(np.mean(mse_l))
print(np.mean(r2_l))

Fold 1


In [ ]:
mse_l = []
r2_l = []
for y_true, preds in fold_results:
    mse_l.append(mean_squared_error(y_true, preds))
    r2_l.append(r2_score(y_true, preds))

print(np.mean(mse_l))
print(np.mean(r2_l))

In [ ]:
#Training

# import torch.optim as optim
# from sklearn.model_selection import KFold
# criterion = nn.MSELoss()
# optimizer = optim.Adam(net.parameters(), lr=5e-4)
#
#
# epochs = 100
# for epoch in range(epochs):
#     net.train()
#     train_loss = 0.0
#     for batch_X, batch_y in train_loader:
#         batch_X = batch_X.to(device)
#         batch_y = batch_y.to(device).float().unsqueeze(1)
#
#         optimizer.zero_grad()
#         outputs = net(batch_X)
#         loss = criterion(outputs, batch_y)
#         loss.backward()
#         optimizer.step()
#
#         train_loss += loss.item()
#
#     train_loss /= len(train_loader)
#
#     net.eval()
#     val_loss = 0.0
#     with torch.no_grad():
#         for batch_X, batch_y in val_loader:
#             batch_X = batch_X.to(device)
#             batch_y = batch_y.to(device).float().unsqueeze(1)
#             val_outputs = net(batch_X)
#             loss = criterion(val_outputs, batch_y)
#             val_loss += loss.item()
#
#     val_loss /= len(val_loader)
#
#     print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}\n")


In [ ]:
import matplotlib.pyplot as plt
plt.plot(x_train[10])
plt.show()

In [ ]:


with torch.no_grad():
    outputs = net(x_test.to(device))

preds = outputs.cpu().numpy().flatten()
y_true = y_test.cpu().numpy().flatten()

plt.scatter(range(len(y_true)), y_true, label="True", color="blue")
plt.scatter(range(len(preds)), preds, label="Predicted", color="red")
plt.xlabel("Sample index")
plt.ylabel("Value")
plt.legend()
plt.show()
